# Compare aligned audio segments
Compare music similarity between similar structural segments by hierarchically decomposing structure and finding segment alignment



## > Library importing

In [ ]:
#Computation
import numpy as np
import scipy
from scipy.interpolate import interp2d
import matplotlib.pyplot as plt
import cv2

#Data Processing
import sklearn.cluster

#Audio
import librosa
from librosa import display

#System
import glob
import os
import sys

#Pickling
import dill

## > Loading audio

In [ ]:
#Choose directory containing audiofiles
directory = 'test_dataset'

In [ ]:
#Read all paths in specified directory
all_filepaths = []
all_names= []
for root, dirs, files in os.walk(directory):
        for name in files:
            if (('.wav' in name) or ('.aif' in name) or ('.mp3' in name)):
                filepath = os.path.join(root, name)
                all_filepaths.append(filepath)
                all_names.append(name[:-4])

#Load all audiofiles and store in array
all_audio = []
file_no = len(all_filepaths)
for f in range(file_no):
    y, sr = librosa.load(all_filepaths[f], sr=22050, mono=True)
    all_audio.append((y, sr))
    sys.stdout.write("\rLoaded %i/%s pieces." % ((f+1), str(file_no)))
    sys.stdout.flush()

## > Compute Laplacian

In [ ]:
BINS_PER_OCTAVE = 12 * 3
N_OCTAVES = 7
all_A = []

for i in range(len(all_dirs)):
    y, sr = librosa.load(all_dirs[i])
    #print('Loaded ' + str(all_dirs[i]))
    C = librosa.amplitude_to_db(np.abs(librosa.cqt(y=y, sr=sr,
                                        bins_per_octave=BINS_PER_OCTAVE,
                                        n_bins=N_OCTAVES * BINS_PER_OCTAVE)),
                            ref=np.max)
    #print('Spectrogram of ' + str(all_dirs[i]))
    tempo, beats = librosa.beat.beat_track(y=y, sr=sr, trim=False)
    #print('Beat tracking ' + str(all_dirs[i]))
    Csync = librosa.util.sync(C, beats, aggregate=np.median)
    #print('Syncing ' + str(all_dirs[i]))
    Cstack = librosa.feature.stack_memory(Csync, 4)
    #print('Stack memory ' + str(all_dirs[i]))
    
    R = librosa.segment.recurrence_matrix(Cstack, width=3, mode='affinity',
                                      sym=True)
    #print('Recurrence matrix ' + str(all_dirs[i]))
    df = librosa.segment.timelag_filter(scipy.ndimage.median_filter)
    Rf = df(R, size=(1, 7))
    #print('Timelag filter ' + str(all_dirs[i]))
    Rf = librosa.segment.path_enhance(Rf, 15)
    #print('Path enhance ' + str(all_dirs[i]))
    
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    Msync = librosa.util.sync(mfcc, beats)

    path_distance = np.sum(np.diff(Msync, axis=1)**2, axis=0)
    sigma = np.median(path_distance)
    path_sim = np.exp(-path_distance / sigma)

    R_path = np.diag(path_sim, k=1) + np.diag(path_sim, k=-1)
    #print('Sequence matrix ' + str(all_dirs[i]))
    deg_path = np.sum(R_path, axis=1)
    deg_rec = np.sum(Rf, axis=1)

    print(deg_path.shape)
    print(deg_rec.shape)
    mu = deg_path.dot(deg_path + deg_rec) / np.sum((deg_path + deg_rec)**2)

    A = mu * Rf + (1 - mu) * R_path
    #print('Balance combination ' + str(all_dirs[i]))
    
    L = scipy.sparse.csgraph.laplacian(A, normed=True)
    
    print('Completed ' + str(all_dirs[i]))
    
print("Completed")

## > Compute Laplacian approximations

## > Spectral Clustering